<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/BubbleDewCalculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.1 MB/s eta 0:00:00


In [54]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from plotly.subplots import make_subplots
from scipy.optimize import root

In [3]:
p=Props(["Benzene","Toluene"])

In [14]:
def bubbleP_ideal(x, T):
    Pi = x*p.Pvap(T)
    P = np.sum(Pi)
    return P, Pi/P

In [15]:
z=np.array([0.3, 0.7])
z

array([0.3, 0.7])

In [52]:
bubbleP_ideal(z, 373.15)

(Array(179893.71528287, dtype=float64), Array([1., 0.], dtype=float64))

In [35]:
def bubbleT_ideal(x, P):

    def P_dev(T):
        return bubbleP_ideal(x,T)[0] - P

    T=root(P_dev, 300.).x[0]
    return T,x*p.Pvap(T)/P


In [36]:
bubbleT_ideal(z, 101325.)

(371.6543967389883, Array([0.51137952, 0.48862048], dtype=float64))

In [22]:
def dewP_ideal(y, T):
    P=1./(np.sum(y/p.Pvap(T)))
    return P, y*P/p.Pvap(T)

In [23]:
dewP_ideal(z, 373.15)

(Array(89901.78980933, dtype=float64),
 Array([0.14992484, 0.85007516], dtype=float64))

In [37]:

def dewT_ideal(y, P):

    def P_dev(T):
        return dewP_ideal(y, T)[0] - P
    T = root(P_dev, 300.).x[0]

    return T,  y*P/p.Pvap(T)

In [38]:
dewT_ideal(z, 89901.79)

(373.15000007098485, Array([0.14992484, 0.85007516], dtype=float64))

In [65]:
z1 = np.linspace(0.,1.,20)
zs=np.c_[z1, 1-z1]

In [72]:
x1s=[]
y1s=[]
Ps=[]
T=373.15
for z in zs:
    P, y =bubbleP_ideal(z, T)
    x1s.append(z[0])
    y1s.append(y[0])
    Ps.append(P)

fig=make_subplots()
fig.add_scatter(x=x1s, y=Ps, name='Bubble')
fig.add_scatter(x=y1s, y=Ps, name='Dew')
fig.update_layout(width=600, height=400, template='plotly_dark',
                  title=f'Benzene (1)/Toluene (2) T={T}')

In [74]:
x1s=[]
y1s=[]
Ts=[]
P=101325.
for z in zs:
    T, y =bubbleT_ideal(z, P)
    x1s.append(z[0])
    y1s.append(y[0])
    Ts.append(T)

fig=make_subplots()
fig.add_scatter(x=x1s, y=Ts, name='Bubble')
fig.add_scatter(x=y1s, y=Ts, name='Dew')
fig.update_layout(width=600, height=400, template='plotly_dark',
                  title=f'Benzene (1)/Toluene (2) P={P}')

In [80]:
fig3=make_subplots()
fig3.add_scatter(x=x1s, y=y1s)
fig3.add_scatter(x=[0.,1.],y=[0.,1.], mode='lines', line_color='rgb(50,50,50)')
fig3.update_layout(width=400,height=400,template='plotly_dark', showlegend=False)